# Session 3 - Maîtriser l'Attention (CORRECTION)

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Implémenter l'attention de A à Z et découvrir le Multi-Head

---

## Objectifs pédagogiques

À la fin de cette session, vous serez capable de :
1. Implémenter la fonction `scaled_dot_product_attention()` complète
2. Créer une classe `SelfAttention` réutilisable en PyTorch
3. Visualiser et interpréter l'attention d'un vrai modèle
4. Comprendre pourquoi on utilise **plusieurs têtes** d'attention

---

## Rappel : Où en sommes-nous ?

Dans la session précédente, vous avez :
- ✅ Compris le **Positional Encoding** (encoder l'ordre des mots)
- ✅ Vu le lien entre **similarité** et **produit scalaire**
- ✅ Découvert les concepts de **Query, Key, Value**
- ✅ Calculé les scores d'attention étape par étape

Aujourd'hui, on passe à l'**implémentation complète** !

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy transformers -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

## 1. Rappel : La formule de l'attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

| Étape | Opération | Rôle |
|-------|-----------|------|
| 1 | $QK^T$ | Calculer les scores de similarité |
| 2 | $\div \sqrt{d_k}$ | Stabiliser les gradients |
| 3 | softmax | Transformer en probabilités |
| 4 | $\times V$ | Moyenne pondérée des values |

Dans les exercices 1-4, vous avez fait ces étapes **séparément**. Maintenant, on les **combine** dans une fonction.

---

## 2. Exercice 1 : Fonction d'attention complète

Regroupez les 4 étapes dans une seule fonction réutilisable.

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (..., seq_len, d_k)
        K: Keys, shape (..., seq_len, d_k)
        V: Values, shape (..., seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention, même shape que V
        attention_weights: Poids d'attention, shape (..., seq_len, seq_len)
    """
    # CORRECTION 1: Récupérer d_k (dernière dimension de K)
    d_k = K.shape[-1]
    
    # CORRECTION 2: Calculer les scores QK^T
    scores = Q @ K.transpose(-2, -1)
    
    # CORRECTION 3: Appliquer le scaling (diviser par sqrt(d_k))
    scaled_scores = scores / math.sqrt(d_k)
    
    # CORRECTION 4: Appliquer softmax sur la dernière dimension
    attention_weights = F.softmax(scaled_scores, dim=-1)
    
    # CORRECTION 5: Calculer la sortie (weights @ V)
    output = attention_weights @ V
    
    return output, attention_weights

In [ ]:
# Test de votre fonction
Q_test = torch.randn(4, 8)  # 4 tokens, dimension 8
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Output shape: {output.shape}")    # Attendu: (4, 8)
print(f"Weights shape: {weights.shape}")  # Attendu: (4, 4)
print(f"Somme des poids par ligne: {weights.sum(dim=-1)}")  # Attendu: [1, 1, 1, 1]

### Vérification : Effet du scaling

Pourquoi diviser par $\sqrt{d_k}$ ? Voyons l'effet.

In [ ]:
# Comparaison avec et sans scaling
d_k_grand = 512  # Dimension typique dans un Transformer

Q_grand = torch.randn(10, d_k_grand)
K_grand = torch.randn(10, d_k_grand)

# Sans scaling
scores_sans = Q_grand @ K_grand.T
attention_sans = F.softmax(scores_sans, dim=-1)

# Avec scaling
scores_avec = (Q_grand @ K_grand.T) / math.sqrt(d_k_grand)
attention_avec = F.softmax(scores_avec, dim=-1)

print("=== SANS SCALING ===")
print(f"Scores - min: {scores_sans.min():.1f}, max: {scores_sans.max():.1f}")
print(f"Attention max par ligne: {attention_sans.max(dim=-1).values[:5]}")

print("\n=== AVEC SCALING ===")
print(f"Scores - min: {scores_avec.min():.1f}, max: {scores_avec.max():.1f}")
print(f"Attention max par ligne: {attention_avec.max(dim=-1).values[:5]}")

print("\n✅ Avec scaling, l'attention est mieux répartie (pas de valeur proche de 1)")

---

## 3. Exercice 2 : Classe SelfAttention

### D'où viennent Q, K, V ?

Jusqu'ici, on a utilisé des tenseurs aléatoires. En pratique, **Q, K, V sont calculés à partir des embeddings** via des matrices apprenables.

```
x (embeddings) ──┬──► W_q ──► Q   (ce que je cherche)
                 ├──► W_k ──► K   (mon identité)
                 └──► W_v ──► V   (mon contenu)
```

**Pourquoi 3 matrices différentes ?**

Un même mot a besoin de **3 représentations** selon son rôle :
- **Query** : "Qu'est-ce que je cherche ?"
- **Key** : "Comment les autres me voient ?"
- **Value** : "Quelle information je transmets ?"

Ces matrices sont **apprises** pendant l'entraînement.

In [ ]:
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    Projette l'input x vers Q, K, V puis applique l'attention.
    """
    
    def __init__(self, embed_dim):
        """
        Args:
            embed_dim: Dimension des embeddings d'entrée
        """
        super().__init__()
        self.embed_dim = embed_dim
        
        # CORRECTION 1: Créer 3 couches linéaires pour projeter vers Q, K, V
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)
        
        Returns:
            output: Résultat de l'attention
            attention_weights: Poids d'attention
        """
        # CORRECTION 2: Projeter x vers Q, K, V
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # CORRECTION 3: Appliquer la fonction scaled_dot_product_attention
        output, attention_weights = scaled_dot_product_attention(Q, K, V)
        
        return output, attention_weights

In [ ]:
# Test du module
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")   # Attendu: (2, 5, 32)
print(f"Weights shape: {weights.shape}") # Attendu: (2, 5, 5)

# Compter les paramètres
n_params = sum(p.numel() for p in attention_layer.parameters())
print(f"\nNombre de paramètres: {n_params:,}")

---

## 4. Visualisation sur un vrai modèle

Maintenant qu'on a compris et implémenté l'attention, regardons ce que ça donne sur un modèle **réellement entraîné**.

### Tokens spéciaux

Les modèles BERT ajoutent des tokens spéciaux :

| Token | Rôle |
|-------|------|
| **[CLS]** | Début de phrase. Son vecteur représente toute la phrase. |
| **[SEP]** | Fin de phrase / séparateur. |

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Charger DistilBERT
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)
model.eval()

# Phrase de test (en anglais pour ce modèle)
phrase = "The cat sat on the mat because it was tired"

# Tokeniser
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions

print(f"Phrase: {phrase}")
print(f"Tokens: {tokens}")
print(f"\nNombre de couches: {len(attentions)}")
print(f"Nombre de têtes par couche: {attentions[0].shape[1]}")

In [ ]:
# Visualiser l'attention d'une tête spécifique
# Couche 5, Tête 2 : capture bien la coréférence "it" → "cat"
layer = 4
head = 1

attention_matrix = attentions[layer][0, head].numpy()

plt.figure(figsize=(10, 8))
plt.imshow(attention_matrix, cmap='Blues')
plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
plt.yticks(range(len(tokens)), tokens)
plt.xlabel("Tokens regardés (Keys)")
plt.ylabel("Tokens qui regardent (Queries)")
plt.title(f"Attention réelle - Couche {layer+1}, Tête {head+1}")
plt.colorbar(label="Poids d'attention")

for i in range(len(tokens)):
    for j in range(len(tokens)):
        val = attention_matrix[i, j]
        plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                color='white' if val > 0.3 else 'black', fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
# Que regarde le pronom "it" ?
it_index = tokens.index("it")

print(f"Attention de 'it' (Couche {layer+1}, Tête {head+1}) :")
print("-" * 40)

for token, weight in zip(tokens, attention_matrix[it_index]):
    bar = "█" * int(weight * 30)
    highlight = " ← antécédent !" if token == "cat" else ""
    print(f"  {token:10} {weight:.2f} {bar}{highlight}")

**Observation** : Le pronom "it" regarde principalement "cat" → le modèle a appris la **coréférence** !

Mais cette tête ne capture qu'**un type de relation**. Pour capturer plusieurs types de relations (syntaxe, sémantique, proximité...), on utilise **plusieurs têtes**.

---

## 5. Pourquoi Multi-Head Attention ?

### Le problème avec une seule tête

Une seule tête d'attention calcule **une** représentation des relations entre mots.

Mais dans une phrase, il y a **plusieurs types de relations** :
- Relations **syntaxiques** (sujet-verbe)
- Relations **sémantiques** (sens, coréférence)
- Relations de **proximité** (mots voisins)
- etc.

### La solution : plusieurs têtes

Chaque tête peut apprendre à détecter un type de relation différent !

**Analogie** : C'est comme avoir plusieurs experts qui analysent une phrase sous différents angles, puis combinent leurs analyses.

In [ ]:
# Illustration : différentes têtes capturent différentes relations
phrase_demo = ["Le", "chat", "noir", "mange", "la", "souris"]

# Tête 1 : Relations syntaxiques (sujet-verbe)
attention_syntaxe = torch.tensor([
    [0.3, 0.5, 0.1, 0.05, 0.03, 0.02],
    [0.1, 0.3, 0.1, 0.4, 0.05, 0.05],
    [0.1, 0.6, 0.2, 0.05, 0.03, 0.02],
    [0.05, 0.5, 0.05, 0.2, 0.1, 0.1],
    [0.02, 0.03, 0.02, 0.03, 0.3, 0.6],
    [0.02, 0.1, 0.02, 0.4, 0.06, 0.4],
])

# Tête 2 : Relations de proximité
attention_proximite = torch.tensor([
    [0.5, 0.4, 0.08, 0.01, 0.005, 0.005],
    [0.3, 0.4, 0.25, 0.04, 0.005, 0.005],
    [0.1, 0.35, 0.35, 0.15, 0.03, 0.02],
    [0.02, 0.1, 0.3, 0.35, 0.18, 0.05],
    [0.01, 0.02, 0.05, 0.25, 0.4, 0.27],
    [0.005, 0.01, 0.02, 0.1, 0.35, 0.515],
])

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (attn, title) in enumerate([
    (attention_syntaxe, "Tête 1 : Relations syntaxiques\n(chat → mange)"),
    (attention_proximite, "Tête 2 : Proximité\n(mots voisins)")
]):
    ax = axes[idx]
    im = ax.imshow(attn, cmap='Blues')
    ax.set_xticks(range(6))
    ax.set_xticklabels(phrase_demo, rotation=45)
    ax.set_yticks(range(6))
    ax.set_yticklabels(phrase_demo)
    ax.set_title(title, fontsize=12)
    plt.colorbar(im, ax=ax)

plt.suptitle("Chaque tête capture des relations différentes", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## 6. Architecture Multi-Head Attention

### Schéma

```
Input (seq_len, embed_dim)
        ↓
   ┌────┴────┬────────┬────────┐
   ↓         ↓        ↓        ↓
 Head 1   Head 2   Head 3   Head 4
   ↓         ↓        ↓        ↓
   └────┬────┴────────┴────────┘
        ↓
    Concat
        ↓
   Linear (W_o)
        ↓
Output (seq_len, embed_dim)
```

### Dimensions

| Paramètre | Exemple | Description |
|-----------|---------|-------------|
| embed_dim | 512 | Dimension des embeddings |
| num_heads | 8 | Nombre de têtes |
| d_k | 64 | Dimension par tête = embed_dim / num_heads |

Chaque tête travaille avec une **dimension réduite** ($d_k$), puis on **concatène** les résultats.

## 7. Exercice 3 : split_heads (Multi-Head)

La première étape du Multi-Head est de **séparer les têtes**.

On transforme `(batch, seq_len, embed_dim)` en `(batch, num_heads, seq_len, d_k)`

In [ ]:
# Configuration
batch_size = 2
seq_len = 6
embed_dim = 32
num_heads = 4
d_k = embed_dim // num_heads  # 32 / 4 = 8

print(f"embed_dim: {embed_dim}")
print(f"num_heads: {num_heads}")
print(f"d_k (dim par tête): {d_k}")

# Input
x = torch.randn(batch_size, seq_len, embed_dim)
print(f"\nInput shape: {x.shape}")

In [ ]:
def split_heads(x, num_heads):
    """
    Sépare les têtes d'attention.
    
    Reshape: (batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)
    
    Args:
        x: Tensor de shape (batch, seq_len, embed_dim)
        num_heads: Nombre de têtes
    
    Returns:
        Tensor de shape (batch, num_heads, seq_len, d_k)
    """
    batch_size, seq_len, embed_dim = x.shape
    d_k = embed_dim // num_heads
    
    # CORRECTION:
    # Étape 1: Séparer embed_dim en (num_heads, d_k)
    x = x.view(batch_size, seq_len, num_heads, d_k)
    # Étape 2: Réorganiser pour avoir num_heads en position 1
    x = x.transpose(1, 2)
    
    return x

In [ ]:
# Test
x_test = torch.randn(2, 6, 32)  # batch=2, seq=6, embed=32
x_heads = split_heads(x_test, num_heads=4)

print(f"Avant split: {x_test.shape}")   # (2, 6, 32)
print(f"Après split: {x_heads.shape}")  # Attendu: (2, 4, 6, 8)

if x_heads is not None and x_heads.shape == (2, 4, 6, 8):
    print("\n✅ Correct !")
else:
    print("\n❌ Vérifiez votre implémentation")

---

## 8. Récapitulatif

### Ce que nous avons appris aujourd'hui

| Concept | Ce qu'on a fait |
|---------|----------------|
| **Exercice 5** | Fonction `scaled_dot_product_attention()` complète |
| **Exercice 6** | Classe `SelfAttention` avec projections W_q, W_k, W_v |
| **Visualisation** | Voir ce que l'attention capture sur DistilBERT |
| **Multi-Head** | Comprendre pourquoi plusieurs têtes |
| **split_heads** | Première étape de l'implémentation Multi-Head |

### Prochaine session

On va :
1. Terminer l'implémentation Multi-Head (`concat_heads`, classe complète)
2. Ajouter le **Feed-Forward Network**
3. Comprendre le **masque causal** (GPT vs BERT)
4. Assembler un **bloc Transformer** complet !

---

## 9. Pour aller plus loin (optionnel)

### Visualiser plusieurs têtes de DistilBERT

In [ ]:
# Visualiser 4 têtes différentes de la même couche
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

heads_to_show = [0, 1, 5, 10]
layer = 4

for idx, head in enumerate(heads_to_show):
    ax = axes[idx // 2, idx % 2]
    w = attentions[layer][0, head].numpy()
    
    im = ax.imshow(w, cmap='Blues')
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=8)
    ax.set_yticks(range(len(tokens)))
    ax.set_yticklabels(tokens, fontsize=8)
    ax.set_title(f"Tête {head + 1}", fontsize=11)
    plt.colorbar(im, ax=ax)

plt.suptitle(f"Différentes têtes de la couche {layer+1}", fontsize=13)
plt.tight_layout()
plt.show()

# Analyse : que regarde "it" selon chaque tête ?
print(f"\nQue regarde 'it' selon chaque tête ?")
print("=" * 50)

for head in heads_to_show:
    weights = attentions[layer][0, head, it_index].numpy()
    top_idx = weights.argsort()[-2:][::-1]  # Top 2
    print(f"Tête {head+1:2d}: ", end="")
    for i in top_idx:
        print(f"{tokens[i]} ({weights[i]:.2f})  ", end="")
    print()